## Ontology Alignment

In this Tutorial we will align the taxonomy created in Tutorial 6 with an upper ontology

In [1]:
!pip install owlready2


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Let us first load an upper ontology, here the classes of bfo, and list its content

In [2]:
from owlready2 import *

onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()

list(onto.classes())

[owl.Thing,
 DUL.Agent,
 DUL.SocialAgent,
 DUL.Entity,
 DUL.Concept,
 DUL.Collection,
 DUL.SocialObject,
 DUL.InformationRealization,
 DUL.Object,
 DUL.Description,
 DUL.Role,
 DUL.Task,
 DUL.Action,
 DUL.InformationObject,
 DUL.Parameter,
 DUL.Event,
 DUL.Situation,
 DUL.Quality,
 DUL.Region,
 DUL.TimeInterval,
 DUL.Abstract,
 DUL.Amount,
 DUL.PhysicalAttribute,
 DUL.SpaceRegion,
 DUL.BiologicalObject,
 DUL.PhysicalBody,
 DUL.ChemicalObject,
 DUL.Classification,
 DUL.TimeIndexedRelation,
 DUL.Collective,
 DUL.CollectiveAgent,
 DUL.Community,
 DUL.Configuration,
 DUL.Contract,
 DUL.Design,
 DUL.DesignedArtifact,
 DUL.PhysicalArtifact,
 DUL.DesignedSubstance,
 DUL.FunctionalSubstance,
 DUL.Diagnosis,
 DUL.EventType,
 DUL.FormalEntity,
 DUL.Substance,
 DUL.Goal,
 DUL.Group,
 DUL.Plan,
 DUL.InformationEntity,
 DUL.PhysicalObject,
 DUL.LocalConcept,
 DUL.Method,
 DUL.Narrative,
 DUL.NaturalPerson,
 DUL.Person,
 DUL.PhysicalAgent,
 DUL.Norm,
 DUL.ObjectAggregate,
 DUL.Organism,
 DUL.Organiz

#### Let us get the class relations

For this, we create a second namespace for the loaded ontology (bfo), and list subclasses

We can do this until we reach a class that might be a fit as upper class

In [3]:
dul = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
                    
list(dul.Entity.subclasses())

[DUL.Object,
 DUL.Event,
 DUL.Situation,
 DUL.Quality,
 DUL.Abstract,
 DUL.InformationEntity]

#### Now we reached a point where I am unsure what class is suitable to be the superclass of the "Types" class of our taxonomy. In such situations, we can access the ontologie to get the class comment (rdfs.comment annotation property) 

Note that the rdfs namespace is automatically loaded just as well as the owl namespace, which makes things easier.

In [4]:
print(dul.Quality.comment)

["Any aspect of an Entity (but not a part of it), which cannot exist without that Entity. For example, the way the surface of a specific PhysicalObject looks like, or the specific light of a place at a certain time, are examples of Quality, while the encoding of a Quality into e.g. a PhysicalAttribute should be modeled as a Region. \nFrom the design viewpoint, the Quality-Region distinction is useful only when individual aspects of an Entity are considered in a domain of discourse. \nFor example, in an automotive context, it would be irrelevant to consider the aspects of car windows for a specific car, unless the factory wants to check a specific window against design parameters (anomaly detection). \nOn the other hand, in an antiques context, the individual aspects for a specific piece of furniture are a major focus of attention, and may constitute the actual added value, because the design parameters for old furniture are often not fixed, and may not be viewed as 'anomalies'."]


#### At this point I personally decide to stop my search. Our goal is to align our "Types" from the pokemon type taxonomy to the DUL top level ontology. I argue that the type of a pokemon is a quality.. Therefore, we should align our types to be a subclass of the quality class.

### In a second step we want to also load our product taxonomy created in Tutorial 6 and align it with the DUL ontology

#### For this, please upload the saved ontology to this jupyter notebook (if you are using the dockerized version).

The ontology alignment in this case is to assign the product class as a subclass of the DUL.PhysicalObject class

#### To load the file, we now have to use second name. Print the classes to make sure you loaded correctly

In [5]:
prod = get_ontology("./alignmentexercise/TypeTaxonomyFromBulbagarden.owl").load()

list(prod.classes())

[TypeTaxonomyFromBulbagarden.Types,
 TypeTaxonomyFromBulbagarden.Normal,
 TypeTaxonomyFromBulbagarden.Fire,
 TypeTaxonomyFromBulbagarden.Fighting,
 TypeTaxonomyFromBulbagarden.Water,
 TypeTaxonomyFromBulbagarden.Flying,
 TypeTaxonomyFromBulbagarden.Grass,
 TypeTaxonomyFromBulbagarden.Poison,
 TypeTaxonomyFromBulbagarden.Electric,
 TypeTaxonomyFromBulbagarden.Ground,
 TypeTaxonomyFromBulbagarden.Psychic,
 TypeTaxonomyFromBulbagarden.Rock,
 TypeTaxonomyFromBulbagarden.Ice,
 TypeTaxonomyFromBulbagarden.Bug,
 TypeTaxonomyFromBulbagarden.Dragon,
 TypeTaxonomyFromBulbagarden.Ghost,
 TypeTaxonomyFromBulbagarden.Dark,
 TypeTaxonomyFromBulbagarden.Steel,
 TypeTaxonomyFromBulbagarden.Fairy,
 TypeTaxonomyFromBulbagarden.Stellar,
 TypeTaxonomyFromBulbagarden.???]

#### Let us again assign a new namespace for the type taxonomy

In [6]:
tax = get_namespace("http://TypeTaxonomyFromBulbagarden.owl#")

print(tax.Types.iri)

http://TypeTaxonomyFromBulbagarden.owl#Types


## Now on to actually aligning the ontologies!

#### This can now be easily done by aissgning the types class from the product taxonomy file as a subclass of the dul upper ontology.

In [7]:
with onto: 
    tax.Types.is_a.append(dul.Quality)

list(dul.Quality.subclasses())

[TypeTaxonomyFromBulbagarden.Types]

### Unfortunately, we need to do this for all subclasses of the types class as well.

In [12]:
for i in prod.Product.subclasses():
    with onto:
        i.is_a.append(prod.Product)

AttributeError: 'NoneType' object has no attribute 'subclasses'

#### In a last step, let us save the aligned ontology

In [10]:
onto.save(file = "AlignedOntology.owl", format = "rdfxml")